In [ ]:
import cv2
import os
import re
import csv

# Define classes and key mappings
classes = ['scissors', 'forceps', 'gauze']
key_to_class = {'s': 'scissors', 'f': 'forceps', 'g': 'gauze'}

# Initialize class counters
class_counters = {'scissors': 0, 'forceps': 0, 'gauze': 0}

# Set the folder to save images
save_folder = 'a_dataset_images'
if not os.path.exists(save_folder):
    os.makedirs(save_folder)

# Initialize image_counter
image_counter = 1

# Initialize labels dictionary
labels = {}

labels_file = os.path.join(save_folder, 'labels.csv')
if os.path.exists(labels_file):
    # Read existing labels
    with open(labels_file, 'r') as f:
        reader = csv.reader(f)
        for row in reader:
            filename, label = row
            labels[filename] = label
            class_counters[label] += 1
            num = int(re.findall(r'\d+', filename)[0])
            if num >= image_counter:
                image_counter = num +1

# Now, image_counter is set, and class_counters are updated

current_class = 'scissors'  # default class

# Open the camera
cap = cv2.VideoCapture(1)  # Use 0 for default camera

if not cap.isOpened():
    print("Cannot open camera")
    exit()

print("Starting image collection.")
print("Press Spacebar to take a picture.")
print("Press 's' for scissors, 'f' for forceps, 'g' for gauze to switch class.")
print("Press 'q' to quit.")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Display the current class and counts on the frame
        info_text = f"Class: {current_class} | Counts - scissors: {class_counters['scissors']}, forceps: {class_counters['forceps']}, gauze: {class_counters['gauze']}"
        cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

        cv2.imshow('Camera', frame)

        key = cv2.waitKey(1) & 0xFF

        if key == ord('q'):
            print("Exiting program.")
            break
        elif key == 32:  # Spacebar pressed
            # Save image
            filename = f'img{image_counter}.jpg'
            filepath = os.path.join(save_folder, filename)
            cv2.imwrite(filepath, frame)
            print(f"Saved {filename}")

            # Update labels dictionary
            labels[filename] = current_class

            # Write to labels.csv
            with open(labels_file, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([filename, current_class])

            image_counter += 1
            class_counters[current_class] +=1

        elif key in [ord('s'), ord('f'), ord('g')]:
            current_class = key_to_class[chr(key)]
            print(f"Switched to class: {current_class}")

except KeyboardInterrupt:
    print("Interrupted by user.")

# Release the camera and close windows
cap.release()
cv2.destroyAllWindows()
